In [59]:
from pulp import LpProblem, LpVariable, LpMaximize, LpMinimize, LpStatus, lpSum, value, apis

# from pulp import pulpTestAll
# pulpTestAll()



# 1. Initialise model
model = LpProblem("Maximize LP", LpMaximize)

# 2. Define Decision Variables: 
x1 = LpVariable('X1', lowBound=0, upBound=None, cat="Continuous")
x2 = LpVariable('X2', lowBound=0, upBound=None, cat="Continuous")
x3 = LpVariable('X3', lowBound=0, upBound=None, cat="Continuous")
x4 = LpVariable('X4', lowBound=0, upBound=None, cat="Continuous")

# 3. Define objective function: 
model += lpSum(2 * x1 + 4 * x2 + 3 * x3 + x4), 'obj'

# 4. Define constraints: our model's limitations
model += lpSum(3 * x1 + x2 +x3 + 4 * x4) <= 12, 'c1'
model += lpSum(x1 -3*x2 + 2*x3 + 3 * x4) <= 7, 'c2'  
model += lpSum(2*x1 + x2 + 3*x3 - x4) <= 10, 'c3'
        

# 5. Solve model
model.solve(apis.PULP_CBC_CMD(msg=0))

print("X1 = {} ".format(x1.varValue))
print("X2 = {} ".format(x2.varValue))
print("X3 = {} ".format(x3.varValue))
print("X4 = {} ".format(x4.varValue))
print(f"Optimal value of the function: {(x1.varValue * 2) + (x2.varValue * 4) + (x3.varValue * 3) + (x4.varValue)}")


X1 = 0.0 
X2 = 10.4 
X3 = 0.0 
X4 = 0.4 
Optimal value of the function: 42.0


## Change of an objective function coefficient

1. Variable is non-basic (it takes a value of zero in the solution). For example, let us modify the

In [60]:
model.constraints['c1'][x1]
oldvalue = model.objective[x1]
model.objective[x1] = 10
model.solve()
print("X1 = {} ".format(x1.varValue))
print("X2 = {} ".format(x2.varValue))
print("X3 = {} ".format(x3.varValue))
print("X4 = {} ".format(x4.varValue))
print(f"Optimal value of the function: {(x1.varValue * 2) + (x2.varValue * 4) + (x3.varValue * 3) + (x4.varValue)}")
model.objective[x1] = oldvalue  # return to original value

X1 = 2.0 
X2 = 6.0 
X3 = 0.0 
X4 = 0.0 
Optimal value of the function: 28.0


2. Changing the RHS of a constraint

In [61]:

model.constraints['c1'] = lpSum(3 * x1 + x2 +x3 + 4 * x4) <= 13
model.solve()
print("X1 = {} ".format(x1.varValue))
print("X2 = {} ".format(x2.varValue))
print("X3 = {} ".format(x3.varValue))
print("X4 = {} ".format(x4.varValue))
print(f"Optimal value of the function: {(x1.varValue * 2) + (x2.varValue * 4) + (x3.varValue * 3) + (x4.varValue)}")

X1 = 0.0 
X2 = 10.6 
X3 = 0.0 
X4 = 0.6 
Optimal value of the function: 43.0
